A hydrostatic, inviscid, adiabatic, and moist system in isentropic coordinates
==

In [ ]:
from datetime import datetime, timedelta
import gridtools as gt
import numpy as np
import os
from sympl import DataArray
import tasmania as taz

# For interactive plotting
from IPython import display
%matplotlib nbagg

Build the computational grid and get the initial state
--

In [ ]:
domain_x, nx = DataArray([0., 500.], dims='x', attrs={'units': 'km'}).to_units('m'), 26
domain_y, ny = DataArray([-250., 250.], dims='y', attrs={'units': 'km'}).to_units('m'), 26
domain_z, nz = DataArray([400., 300.], dims='air_potential_temperature', attrs={'units': 'K'}), 60

grid = taz.GridXYZ(domain_x, nx, domain_y, ny, domain_z, nz,
                   topo_type='gaussian', topo_time=timedelta(seconds=1800),
                   topo_kwargs={'topo_max_height': DataArray(1000.0, attrs={'units': 'm'}),
                                'topo_width_x': DataArray(50.0, attrs={'units': 'km'}),
                                'topo_width_y': DataArray(50.0, attrs={'units': 'km'})})

time          = datetime(year=1992, month=2, day=20)
x_velocity    = DataArray(15., attrs={'units': 'm s^-1'})
y_velocity    = DataArray(0., attrs={'units': 'm s^-1'})
brunt_vaisala = DataArray(0.01, attrs={'units': 's^-1'})

state = taz.get_default_isentropic_state(grid, time, x_velocity, y_velocity, brunt_vaisala, 
                                         moist_on=True, dtype=np.float64)

Initialize the microphysical parameterizations
--

In [ ]:
# The component calculating the microphysical sources/sinks 
# prescribed by the Kessler scheme; neglect the evaporation of
# rain in the subcloud layers
kessler = taz.Kessler(grid, air_pressure_on_interface_levels=True,
                      rain_evaporation=False, backend=gt.mode.NUMPY)

# The component performing the saturation adjustment 
# as prescribed by the Kessler scheme
saturation = taz.SaturationAdjustmentKessler(grid, air_pressure_on_interface_levels=True,
                                             backend=gt.mode.NUMPY)

Instantiate the dynamical core
--

In [ ]:
# Adopt the flux form of the forward Euler time integration method,
# with the horizontal numerical fluxes calculated via the upwind scheme;
# neglect the sedimentation of rain
dycore = taz.IsentropicDynamicalCore(grid, moist_on=True,
                                     time_integration_scheme='rk3cosmo',
                                     horizontal_flux_scheme='fifth_order_upwind',
                                     horizontal_boundary_type='relaxed',
                                     damp_on=True, damp_type='rayleigh', damp_depth=15,
                                     damp_max=0.0002, damp_at_every_stage=False,
                                     smooth_on=True, smooth_type='second_order',
                                     smooth_coeff=0.24, smooth_at_every_stage=False,
                                     adiabatic_flow=True, sedimentation_on=False,
                                     backend=gt.mode.NUMPY, dtype=np.float64)

Create a monitor for interactive visualization
---------------------------------------------------------------

In [ ]:
# The artist generating the left subplot
drawer1_properties = {
    'fontsize': 16, 'cmap_name': 'BuRd', 'cbar_on': True, 
    'cbar_levels': 18, 'cbar_ticks_step': 4, 
    'cbar_center': 15, 'cbar_half_width': 8.5,
    'cbar_orientation': 'horizontal',
    'cbar_x_label': 'Horizontal velocity [m s$^{-1}$]',
    'draw_vertical_levels': True,
    'linecolor': 'black',
}
drawer1 = taz.Contourf(
    grid, 'x_velocity', 'm s^-1', y=13,  
    xaxis_units='km', zaxis_name='height', zaxis_units='km', properties=drawer1_properties,
)
axes1_properties = {
    'fontsize': 16, 'title_left': '$x$-velocity [m s$^{-1}$]',
    'x_label': '$x$ [km]', 'x_lim': [0, 500],
    'y_label': '$z$ [km]', 'y_lim': [0, 14],
    'text': 'y = 0', 'text_loc': 'upper right',
}
plot1 = taz.Plot(drawer1, axes_properties=axes1_properties)

# The drawer and the artist generating the right subplot
drawer2_properties = {
    'fontsize': 16, 'cmap_name': 'Blues', 'cbar_on': True, 
    'cbar_levels': 18, 'cbar_ticks_step': 4, 
    'cbar_orientation': 'horizontal',
    'draw_vertical_levels': True,
    'linecolor': 'black',
}
drawer2 = taz.Contourf(
    grid, 'mass_fraction_of_cloud_liquid_water_in_air', 'g kg^-1', y=13,  
    xaxis_units='km', zaxis_name='height', zaxis_units='km',
    properties=drawer2_properties,
)
axes2_properties = {
    'fontsize': 16, 'title_left': 'Cloud liquid water [g kg$^{-1}$]',
    'x_label': '$x$ [km]', 'x_lim': [0, 500],
    'y_label': '$z$ [km]', 'y_lim': [0, 14],
    'text': 'y = 0', 'text_loc': 'upper right',
}
plot2 = taz.Plot(drawer2, axes_properties=axes2_properties)

# The monitor encompassing and coordinating the two artists
figure_properties = {'fontsize': 16, 'figsize': (12, 7), 'tight_layout': True}
monitor = taz.PlotComposite(
    nrows=1, ncols=2, artists=(plot1, plot2), interactive=False, figure_properties=figure_properties
)

Create a monitor to dump the solution into a NetCDF file
--

In [ ]:
filename = '../tests/baseline_datasets/isentropic_moist_1.nc'
if os.path.exists(filename):
    os.remove(filename)
netcdf_monitor = taz.NetCDFMonitor(filename, grid)
netcdf_monitor.store(state)

Run the simulation
---------------------------

In [ ]:
# Simulation settings
timestep = timedelta(seconds=60)
niter    = 360

for i in range(niter):
    # Update the (time-dependent) topography
    dycore.update_topography((i+1)*timestep)
    
    # Calculate the microphysical tendencies
    tendencies, _ = kessler(state)
    
    # Step the solution
    state_new = dycore(state, tendencies, timestep)
    state.update(state_new)
    
    # Perform the saturation adjustment
    state_new = saturation(state)
    state.update(state_new)

    if (i+1) % 10 == 0:
        # Plot the solution
        plot1.axes_properties['title_right'] = str((i+1)*timestep)
        plot2.axes_properties['title_right'] = str((i+1)*timestep)
        fig = monitor.store((state, state), show=False)
        display.clear_output(wait=True)
        display.display(fig)
        
        # Save the solution
        netcdf_monitor.store(state)
                    
# Write cached states to file
netcdf_monitor.write()
        
print('Simulation successfully completed. HOORAY!')